In [ ]:
# Chase Abram
# IO 2 2021 with Hortacsu

In [211]:
using CSV
using DataFrames
using ForwardDiff
using LinearAlgebra
using Optim

using Random, Distributions
using Statistics

using Plots

In [284]:
# Section 1.3

# Compute the implied probabilities (per logit spec)
function logit_p(p, x, alpha, delta)
    return exp.(alpha*x .+ delta .* reverse(p) .- maximum(p))./(exp(-maximum(p)) .+ exp.(alpha*x .+ delta .* reverse(p) .- maximum(p)))
end

# Finds fixed point (per logit spec)
function fp_logit(x, alpha, delta, sym = false, maxiter = 100, tol = 1e-4)
    if sym
        p = [1/2]
        pnew = [0]
    else
        p = 1/2 .* ones(2)
#         p = [100, 100]
        pnew = zeros(2)
    end
    
    
    f(z) = logit_p(z,x,alpha,delta)
    
    it = 0
    diff = Inf
    
    while it < maxiter && diff > tol
        pnew = f(p)
        diff = abs(maximum(pnew - p))
        it += 1
        p = pnew
        
#         println("it: ", it, ", diff: ", diff, ", p: ", p)
    end
    
#     println("Exited")
#     println("it: ", it, ", diff: ", diff)
#     println("it cond: ", it < maxiter)
#     println("diff cond: ", diff > tol)
    
    if sym
        return p[1]
    end
    
    return p
end



fp_logit (generic function with 4 methods)

In [285]:
alpha = [1, 3]
delta = [1, 6]
x = [1, 2]
p_init = 1/2 .* ones(2)

# logit_p(p_init, x, alpha, delta)

# fp_logit(x, alpha, delta)

for i in 1:2
    for j in 1:2
        println("BNE (sym): alpha = ", alpha[i], ", delta = ", delta[i], ", x = ", x[j])
        println("P_i = ", fp_logit(x[j], alpha[i], delta[i], true))
    end
end
println()
for i in 1:2
    for j in 1:2
        println("BNE: alpha = ", alpha[i], ", delta = ", delta[i], ", x = ", x[j])
        println("P_i = ", fp_logit(x[j], alpha[i], delta[i]))
    end
end

BNE (sym): alpha = 1, delta = 1, x = 1
P_i = 0.8659851186297881
BNE (sym): alpha = 1, delta = 1, x = 2
P_i = 0.9502737662925114
BNE (sym): alpha = 3, delta = 6, x = 1
P_i = 0.9998765126780076
BNE (sym): alpha = 3, delta = 6, x = 2
P_i = 0.9999938555987219

BNE: alpha = 1, delta = 1, x = 1
P_i = [0.8659851186297881, 0.8659851186297881]
BNE: alpha = 1, delta = 1, x = 2
P_i = [0.9502737662925114, 0.9502737662925114]
BNE: alpha = 3, delta = 6, x = 1
P_i = [0.9998765126780076, 0.9998765126780076]
BNE: alpha = 3, delta = 6, x = 2
P_i = [0.9999938555987219, 0.9999938555987219]


In [286]:
alpha = 3
delta = 6
T = 1000
S = 50

Random.seed!(1234)
F = Logistic()
ep = rand(F, S, T, 2)
x = (rand(S, T) .> 1/2) .+ 1
p = fp_logit.(x, alpha, delta, true)
# y = zeros(S,T,2)

y = (alpha .* x .- delta.*p .+ ep) .> 0





50×1000×2 BitArray{3}:
[:, :, 1] =
 1  1  0  1  0  0  0  0  0  0  0  0  1  …  0  0  0  0  0  0  1  1  0  1  0  0
 0  0  1  0  0  0  0  0  0  0  0  1  0     0  1  0  1  0  0  1  0  0  0  0  0
 1  0  1  0  0  1  0  0  1  0  0  0  0     0  0  0  0  0  1  1  0  1  0  1  0
 0  0  0  0  0  0  0  1  0  0  0  0  0     0  0  1  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     1  0  1  0  0  1  0  0  1  0  0  1
 1  0  0  0  0  0  0  0  0  0  0  0  0  …  0  1  0  0  0  1  1  1  0  0  0  0
 0  1  0  0  0  0  1  0  0  0  0  0  0     0  0  0  0  0  1  0  0  0  0  0  0
 0  0  1  1  1  0  1  0  1  1  1  1  1     1  1  1  0  1  0  0  1  0  1  0  1
 0  0  0  0  0  1  0  1  1  1  1  0  0     0  0  0  1  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  1  0  0  0  1  1  0     0  0  0  1  0  1  0  0  0  0  1  0
 0  1  0  0  0  0  0  1  0  0  0  0  0  …  1  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  1  0     0  0  1  0  0  0  0  0  1  0  0  0
 0  1  1  0  0  0  0  0  1  0

In [9]:
# Load data
df = DataFrame()
df = CSV.read("psetTwo.csv", DataFrame)

# Mileage
mileage = df.milage
N = length(mileage)

# Find replacement periods
d_rep = [0; (mileage[2:N] - mileage[1:N-1]) .< 0]

5000-element Array{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 1

In [556]:
# Discretize the domain (equispacing)
# function disc_domain(data, K)
#     M = maximum(data)
#     m = minimum(data)
#     return LinRange(m,M,K)
# end

function disc_domain(data, K)
    data_sorted = sort(data)
    
    disc = zeros(K)
    
    for k in 1:K
        disc[k] = data_sorted[Int(floor(k*N/K))]
    end
    
    return disc
end

# Map value to its chunk
function to_chunk(vs, chunks)
    
    its = zeros(length(vs))
    for j in 1:length(vs)
        it = 1
        while vs[j] > chunks[it] && it < length(chunks)
            it += 1
        end
        its[j] = it
    end
    return Int64.(its)
end

# Create transition matrices
function trans_mats(m, d, K)
    disc = disc_domain(m, K)
    
#     println("disc: ", disc)
    rep = zeros(K,K)
    no_rep = zeros(K,K)
    
    chunk_map = to_chunk(m, disc)
    
    # Fix this for single matrix
    for i in 2:length(m)
        if d[i] > 0
            rep[chunk_map[i-1], chunk_map[i]] += 1
        else
            no_rep[chunk_map[i-1], chunk_map[i]] += 1
        end
    end
    
    for k in 1:K
        if sum(rep[k,:]) == 0
            rep[k,1] = 1
        end
        
        if sum(no_rep[k,:]) == 0
            no_rep[k,k] = 1
        end
        
    end
    
    # Normalize
    rep = rep.*(1 ./max.(sum(rep,dims=2), 1.0))
    no_rep = no_rep.*(1 ./max.(sum(no_rep,dims=2), 1.0))
    
    return rep, no_rep
end

# Utility (shock not included)
function u(x, d, theta)
    if d == 0
        return -theta[1].*x - theta[2].*(x./100).^2
    else
        return -theta[3]
    end
end

function uj(x,d,theta)
    if d == 0
        return hcat(-x, -theta[2]*x/100, zeros(length(x)))
    else
        return hcat(zeros(length(x)), zeros(length(x)), -ones(length(x)))
    end
    
#     return (d .== 0) .* [-x, -theta[2].*x/100, 0] .+ (d .> 0) .* [0, 0, -1]
end

# For fixed point of EV
function Gamma(EV, P, X, beta, theta)
#     println("u : ", u(X,0,theta))
#     println("inside log: ", exp.(u(X,0,theta) .+ beta.*EV) .+ exp.(u(X,1,theta) .+ beta.*EV[1]))
    return P*log.(exp.(u(X,0,theta) .+ beta.*EV) .+ exp.(u(X,1,theta) .+ beta.*EV[1]))
end

# Conditional choice probs
# (sum to 1 when considering exactly exhaustive choices)
function ch_probs(EV, X, beta, theta)
#     d0 = exp.(u(X,0,theta) .+ beta.*EV .- maximum(u(X,0,theta) .+ beta.*EV))./(exp.(u(X,0,theta) .+ beta.*EV .- maximum(u(X,0,theta) .+ beta.*EV)) .+ exp.(u(X,1,theta) .+ beta.*EV[1] .- maximum(u(X,0,theta) .+ beta.*EV)))
#     d1 = exp.(u(X,1,theta) .+ beta.*EV[1])./(exp.(u(X,0,theta) .+ beta.*EV) .+ exp.(u(X,1,theta) .+ beta.*EV[1]))
    return exp.(u(X,0,theta) .+ beta.*EV .- maximum(u(X,0,theta) .+ beta.*EV))./(exp.(u(X,0,theta) .+ beta.*EV .- maximum(u(X,0,theta) .+ beta.*EV)) .+ exp.(u(X,1,theta) .+ beta.*EV[1] .- maximum(u(X,0,theta) .+ beta.*EV)))
end

# Jacobian of Gamma
function Gamma_jac_EV(EV, P, X, beta, theta)
    return P*diagm(ch_probs(EV, X, beta, theta)) .* beta .+ (1 .- ch_probs(EV, X, beta, theta)) .* beta .* [i == 1 for j in 1:length(EV), i in 1:length(EV)]
end

function Gamma_jac_theta(EV, P, X, beta, theta)
    return P*(exp.(u(X,0,theta) .+ beta.*EV .- maximum(u(X,0,theta) .+ beta.*EV)) .* uj(X,0, theta) .+ exp.(u(X,1,theta) .+ beta.*EV[1] .- maximum(u(X,0,theta) .+ beta.*EV)) .* uj(X,1,theta))./(exp.(u(X,0,theta) .+ beta.*EV .- maximum(u(X,0,theta) .+ beta.*EV)) .+ exp.(u(X,1,theta) .+ beta.*EV[1] .- maximum(u(X,0,theta) .+ beta.*EV)))
end

# Contraction mapping to solve EV
function contract_EV(EV, P, X, beta, theta, maxiter = 1000, tol = 1e-5)
    it = 1
    diff = Inf
    EVnew = zeros(length(EV))
    while it < maxiter && diff > tol
#         println("contract it: ", it)
        EVnew = Gamma(EV, P, X, beta, theta)
#         println("EVnew: ", EVnew)
        diff = maximum(abs.(EVnew - EV))
#         println("diff: ", diff)
        it += 1
        EV = EVnew
    end
    println("Contraction done")
    println("    it: ", it)
    println("    diff: ", diff)
    println("    EV: ", EV)
    
    return EV
end

# Newton-Kantorovich to solve EV
function nk_EV(EV, P, X, beta, theta, maxiter = 1000, tol = 1e-14)
    it = 1
    diff = Inf
    EVnew = zeros(length(EV))
    while it < maxiter && diff > tol
#         println("nk it: ", it)
#         println("I - Gj: ", (I - Gamma_jac(EV, P, X, beta, theta)))
#         println("EV - G: ", (EV - Gamma(EV, P, X, beta, theta)))
        EVnew = EV - (I - Gamma_jac_EV(EV, P, X, beta, theta)) \ (EV - Gamma(EV, P, X, beta, theta))
#         println("EVnew: ", EVnew)
        diff = maximum(abs.(EVnew - EV))
#         println("diff: ", diff)
        it += 1
        EV = EVnew
    end
    println("Newton-Kantorovich done")
    println("    it: ", it)
    println("    diff: ", diff)
    println("    EV: ", EV)
    
    return EV
end

# Combine contraction and Newton-Kantorovich
function poly_algo(P, X, beta, theta,
        maxiter_c = 1000, tol_c = 1e-5,
        maxiter_nk = 1000, tol_nk = 1e-14
    )
    EV = zeros(length(X))
    
    EV = contract_EV(EV, P, X, beta, theta, maxiter_c, tol_c)
    EV = nk_EV(EV, P, X, beta, theta, maxiter_nk, tol_nk)
    
    return EV
end

# ch_probs(EV, X, beta, theta)

function log_lik(theta, beta, d, x, EV)
#     println(ch_probs(EV, x, beta, theta))
    return d*log.(1 .- ch_probs(EV, x, beta, theta)) + (1 .- d)*log.(ch_probs(EV, x, beta, theta))
end
    

log_lik (generic function with 1 method)

In [558]:
K = 5
P = trans_mats(mileage, d_rep, K)[2]
X = disc_domain(mileage, K)
beta = 0.999
theta = [0.01,0.02,0.03]

EV = zeros(K)

# for i in 1:5000
#     EV = Gamma(EV, P, X, beta,theta)
# #     println("EV: ", EV)
# end
# println("EV: ", EV)

# println(X)

ch_probs(EV, X, beta, theta)

# [i == 1 for j in 1:5, i in 1:5]
# (1.0 .- ch_probs(EV, X, beta, theta)) .* beta .* [i == 1 for j in 1:length(EV), i in 1:length(EV)]

# Gamma_jac(EV, P, X, beta, theta)

# EV = contract_EV(EV, P, X, beta, theta)
# EV = nk_EV(EV, P, X, beta, theta)

# EV = poly_algo(P, X, beta, theta, 10000, 1e-4, 1000, 1e-14)

# d = [1 0 1 0 0]
# x = [X[1] X[2] X[1] X[2] X[3]]'

# log_lik(theta, beta, d, x, EV)
# ch_probs(EV, x, beta, theta)
# X
uj(x,0,theta)

5×3 Array{Float64,2}:
 -28.0  -0.0056  0.0
 -52.0  -0.0104  0.0
 -28.0  -0.0056  0.0
 -52.0  -0.0104  0.0
 -78.0  -0.0156  0.0

In [487]:
K = 5

disc = disc_domain(mileage, K)

# to_chunk(mileage, disc)

# # plot(to_chunk(mileage, disc))

trans_mats(mileage, d_rep, K)[2]

5×5 Array{Float64,2}:
 0.599607  0.400393  0.0       0.0       0.0
 0.0       0.590447  0.409553  0.0       0.0
 0.0       0.0       0.635551  0.364449  0.0
 0.0       0.0       0.0       0.690722  0.309278
 0.0       0.0       0.0       0.0       1.0